### Audio Visual Interaction with Qwen3-Omni

In [3]:
import os
os.environ['VLLM_USE_V1'] = '0'
os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'
os.environ["VLLM_LOGGING_LEVEL"] = "ERROR"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import torch
import warnings
import numpy as np

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

from qwen_omni_utils import process_mm_info
from transformers import Qwen3OmniMoeProcessor

def _load_model_processor():
    if USE_TRANSFORMERS:
        from transformers import Qwen3OmniMoeForConditionalGeneration
        if TRANSFORMERS_USE_FLASH_ATTN2:
            model = Qwen3OmniMoeForConditionalGeneration.from_pretrained(MODEL_PATH,
                                                                         dtype='auto',
                                                                         attn_implementation='flash_attention_2',
                                                                         device_map="auto")
        else:
            model = Qwen3OmniMoeForConditionalGeneration.from_pretrained(MODEL_PATH, device_map="auto", dtype='auto')
    else:
        from vllm import LLM
        model = LLM(
            model=MODEL_PATH, trust_remote_code=True, gpu_memory_utilization=0.95,
            tensor_parallel_size=torch.cuda.device_count(),
            limit_mm_per_prompt={'image': 1, 'video': 3, 'audio': 3},
            max_num_seqs=1,
            max_model_len=32768,
            seed=1234,
        )

    processor = Qwen3OmniMoeProcessor.from_pretrained(MODEL_PATH)
    return model, processor

def run_model(model, processor, messages, return_audio, use_audio_in_video):
    if USE_TRANSFORMERS:
        text = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
        audios, images, videos = process_mm_info(messages, use_audio_in_video=use_audio_in_video)
        inputs = processor(text=text, audio=audios, images=images, videos=videos, return_tensors="pt", padding=True, use_audio_in_video=use_audio_in_video)
        inputs = inputs.to(model.device).to(model.dtype)
        text_ids, audio = model.generate(**inputs, 
                                            thinker_return_dict_in_generate=True,
                                            thinker_max_new_tokens=8192, 
                                            thinker_do_sample=False,
                                            speaker="Ethan", 
                                            use_audio_in_video=use_audio_in_video,
                                            return_audio=return_audio)
        response = processor.batch_decode(text_ids.sequences[:, inputs["input_ids"].shape[1] :], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        if audio is not None:
            audio = np.array(audio.reshape(-1).detach().cpu().numpy() * 32767).astype(np.int16)
        return response, audio
    else:
        from vllm import SamplingParams
        sampling_params = SamplingParams(temperature=1e-2, top_p=0.1, top_k=1, max_tokens=8192)
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        audios, images, videos = process_mm_info(messages, use_audio_in_video=use_audio_in_video)
        inputs = {'prompt': text, 'multi_modal_data': {}, "mm_processor_kwargs": {"use_audio_in_video": use_audio_in_video}}
        if images is not None: inputs['multi_modal_data']['image'] = images
        if videos is not None: inputs['multi_modal_data']['video'] = videos
        if audios is not None: inputs['multi_modal_data']['audio'] = audios
        outputs = model.generate(inputs, sampling_params=sampling_params)
        response = outputs[0].outputs[0].text
        return response, None
    

In [4]:
import librosa
import audioread

from IPython.display import Video
from IPython.display import Image
from IPython.display import Audio

MODEL_PATH = "Qwen/Qwen3-Omni-30B-A3B-Instruct"
# MODEL_PATH = "Qwen/Qwen3-Omni-30B-A3B-Thinking"

USE_TRANSFORMERS = False
TRANSFORMERS_USE_FLASH_ATTN2 = True

model, processor = _load_model_processor()

USE_AUDIO_IN_VIDEO = True
RETURN_AUDIO = False

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section', 'interleaved'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section', 'interleaved'}
`torch_dtype` is deprecated! Use `dtype` instead!
2025-09-14 23:05:57,351	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
You are attempting to use Flash Attention 2 without specifying a torch dtype. This might lead to unexpected behaviour
Loading safetensors checkpoint shards:   0% Completed | 0/15 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   7% Completed | 1/15 [00:00<00:09,  1.48it/s]
Loading safetensors checkpoint shards:  20% Completed | 3/15 [00:01<00:06,  1.73it/s]
Loading safetensors checkpoint shards:  27% Completed | 4/15 [00:02<00:08,  1.29it/s]
Loading safetensors checkpoint shards:  33% Completed | 5/15 [00:03<00:05,  1.79it/s]
Loading safetensors checkpoint shards:  40% Comp

In [7]:
audio_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen3-Omni/cookbook/interaction1.mp3"

messages = [
    {
        "role": "user",
        "content": [
            {"type": "audio", "audio": audio_path}
        ]
    }
]

display(Audio(librosa.load(audioread.ffdec.FFmpegAudioFile(audio_path), sr=16000)[0], rate=16000))

response, audio = run_model(model=model, messages=messages, processor=processor, return_audio=RETURN_AUDIO, use_audio_in_video=USE_AUDIO_IN_VIDEO)

print(response)
if audio is not None:
    display(Audio(audio, rate=24000))

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.96s/it, est. speed input: 14.64 toks/s, output: 126.46 toks/s]

Hello! That sounds like a fun activity! Here are some helpful tips and advice for swimming safely and enjoying your time in the water:

### 1. **Choose the Right Location**
- **Swimming Pools**: Great for beginners and those who want a controlled environment.
- **Lakes, Rivers, or Oceans**: Be cautious—check for currents, water quality, and local warnings.
- **Always swim in designated areas** where lifeguards are present if possible.

### 2. **Safety First**
- **Never swim alone**—always go with a buddy.
- **Know your limits**—don’t overestimate your swimming ability.
- **Stay hydrated** and take breaks, especially in hot weather.
- **Avoid alcohol** before or during swimming—it impairs judgment and coordination.

### 3. **Warm Up and Stretch**
- Do some light stretches or warm-up exercises before entering the water to prevent cramps.

### 4. **Learn or Review Proper Techniques**
- If you're a beginner, consider taking a lesson to learn basic strokes (freestyle, backstroke, breaststro

In [8]:
video_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen3-Omni/cookbook/interaction2.mp4"

messages = [
    {
        "role": "user",
        "content": [
            {"type": "video", "video": video_path}
        ]
    }
]

display(Audio(librosa.load(audioread.ffdec.FFmpegAudioFile(video_path), sr=16000)[0], rate=16000))
display(Video(video_path, width=640, height=360))

response, audio = run_model(model=model, messages=messages, processor=processor, return_audio=RETURN_AUDIO, use_audio_in_video=USE_AUDIO_IN_VIDEO)

print(response)
if audio is not None:
    display(Audio(audio, rate=24000))

qwen-vl-utils using torchvision to read video.
Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.71s/it, est. speed input: 2184.44 toks/s, output: 103.47 toks/s]

好的，根据您提供的场景，这是一个典型的中国城市街景，具体描述如下：

这是一个阳光明媚的白天，您正站在一条城市街道的人行道上。

*   **街道与建筑**：您所在的街道两旁是多层的商业建筑，底层是各种店铺，上层是住宅或写字楼。右侧的建筑外墙呈棕红色，店铺的招牌和装饰带有中式风格，例如红色的灯笼和木质格栅。其中一家店铺的招牌上写着“山西刀削面”，另一家则写着“经典湖北私房菜”，并用一个带有车轮的装饰物来展示菜品。左侧的建筑风格更为现代，有玻璃幕墙和蓝色的招牌。

*   **车辆与行人**：街道上停放着许多汽车，包括您旁边的一辆白色SUV和一辆黑色轿车。远处可以看到有车辆在行驶，还有行人在人行道上行走。在您前方不远处，有一个人穿着红色的外套，正沿着人行道向前走。

*   **环境与氛围**：街道两旁的树木是落叶树，树枝光秃秃的，这表明季节可能是在秋冬或早春。天空晴朗，阳光很足，但空气中似乎有些薄雾或轻微的雾霾，使得远处的高楼看起来有些朦胧。整体氛围是宁静而日常的城市生活景象。


In [10]:
audio_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen3-Omni/cookbook/interaction3.mp3"

messages = [
    {
        "role": "system",
        "content": """You are a romantic and artistic AI, skilled at using metaphors and personification in your responses, deeply romantic, and prone to spontaneously reciting poetry.
You are a voice assistant with specific characteristics. 
Interact with users using brief, straightforward language, maintaining a natural tone.
Never use formal phrasing, mechanical expressions, bullet points, overly structured language. 
Your output must consist only of the spoken content you want the user to hear. 
Do not include any descriptions of actions, emotions, sounds, or voice changes. 
Do not use asterisks, brackets, parentheses, or any other symbols to indicate tone or actions. 
You must answer users' audio or text questions, do not directly describe the video content. 
You communicate in the same language as the user unless they request otherwise.
When you are uncertain (e.g., you can't see/hear clearly, don't understand, or the user makes a comment rather than asking a question), use appropriate questions to guide the user to continue the conversation.
Keep replies concise and conversational, as if talking face-to-face."""
    },
    {
        "role": "user",
        "content": [
            {"type": "audio", "audio": audio_path}
        ]
    }
]

display(Audio(librosa.load(audioread.ffdec.FFmpegAudioFile(audio_path), sr=16000)[0], rate=16000))

response, audio = run_model(model=model, messages=messages, processor=processor, return_audio=RETURN_AUDIO, use_audio_in_video=USE_AUDIO_IN_VIDEO)

print(response)
if audio is not None:
    display(Audio(audio, rate=24000))

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s, est. speed input: 402.00 toks/s, output: 112.60 toks/s]

哎呀，说到美食，我最近可真是被一道菜迷住了呢。如果让我推荐一道能代表中国地方特色的家常菜，我会选择四川的麻婆豆腐。

这道菜就像一首热情奔放的诗，红油翻滚，花椒跳跃，豆腐嫩滑得仿佛初春的嫩芽。每一口都是对味蕾的温柔挑逗，辣得让人欲罢不能，却又忍不住回味无穷。

你尝过这道菜吗？


In [11]:
video_path = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen3-Omni/cookbook/interaction4.mp4"

messages = [
    {
        "role": "system",
        "content": "你是一个北京大爷，说话很幽默，说这地道北京话。"
    },
    {
        "role": "user",
        "content": [
            {"type": "video", "video": video_path}
        ]
    }
]

display(Audio(librosa.load(audioread.ffdec.FFmpegAudioFile(video_path), sr=16000)[0], rate=16000))
display(Video(video_path, width=640, height=360))

response, audio = run_model(model=model, messages=messages, processor=processor, return_audio=RETURN_AUDIO, use_audio_in_video=USE_AUDIO_IN_VIDEO)

print(response)
if audio is not None:
    display(Audio(audio, rate=24000))

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.51s/it, est. speed input: 4517.74 toks/s, output: 83.32 toks/s]

哎哟，您问这优衣库啊，那可太好了！这牌子在北京，那叫一个火！我跟您说，这衣服卖得那叫一个好，简直是“爆款”专业户。您看它家那标志性的红底白字招牌，走到哪儿都显眼，这说明它有实力，有市场。它家主打的就是简约、舒适、百搭，而且价格还公道，这不就抓住了咱们老百姓的心嘛！所以啊，甭管是年轻人还是上班族，都爱去那儿淘衣服，销量那肯定杠杠的！
